# Skillra PDA: HSE course project

Воспроизводимый аналитический отчёт Skillra о рынке IT-вакансий (ТЗ ВШЭ, этапы 0–4) и продуктовая часть с персонами и skill-gap.

## Вводная
Skillra решает проблему информационного шума на рынке IT-вакансий: кандидаты и джуны тратят недели на поиск релевантных ролей,
а работодатели плохо видят пул талантов и разрывы в навыках. Мы строим Career & Job Market Navigator, опираясь на сырые данные
hh.ru, чтобы проверять продуктовые гипотезы: какие сегменты растут, где есть junior-friendly окна, какие навыки дают премию.

Этот ноутбук — investor story и отчёт по ТЗ ВШЭ: этапы 0–4 показывают полный цикл от парсинга до визуализаций и персон Skillra.
Каждый блок заканчивается выводами: что мы сделали, зачем, что узнали про рынок и как это конвертируется в ценность продукта.

### Команда и набор задач
| Участник                        | Зоны ответственности                                                                                                |
|---------------------------------|---------------------------------------------------------------------------------------------------------------------|
| Адамов Даниил, Полынская Галина | Предобработка (`cleaning.py`), контроль булевых фич и зарплатных инвариантов, тесты.                                |
| Монахов Иван                    | Парсер hh.ru (`parser/hh_scraper.py`), ежедневный сбор данных, обход лимитов hh.ru, документация и быстрая отладка. |
| Полынская Галина, Адамов Даниил | Фичи (`features.py`), витрина рынка (`market.py`), интеграция путей/конфига, пайплайн (`scripts/run_pipeline.py`).  |
| Попов Контантин, Монахов Иван   | EDA и визуализации (`eda.py`, `viz.py`), метрики по ролям/городам/форматам, улучшение графиков и HTML-отчёта.       |
| Монахов Иван, Попов Константин  | Оркестрация шагов плана, чек-лист, поддержка notebook-runner и воспроизводимости, общее ревью решения.              |

### Настройка окружения и загрузка артефактов
Используем функции из `src.skillra_pda`, директории и пути из `config`. Если обработанные данные отсутствуют, запускаем пайплайн `scripts/run_pipeline.py`.

## Этап 0. Парсинг hh.ru и исходные данные
Парсер `parser/hse_vacancies/hh_scraper.py` ежедневно собирает активные IT-вакансии hh.ru по СНГ. Цель — накопить >500k строк за учебный год, чтобы видеть динамику рынка.
Сбор учитывает лимиты hh.ru (пагинация по опыту `EXPERIENCE_SHARDS`, ограничение ~2000 резов на выдачу, дросселирование запросов и ротацию proxy/UA).

### 0.1 Что собирает парсер и как данные используются дальше
- Фильтры: широкая булева строка по IT-ролям (`DEFAULT_QUERY`), регионы СНГ (`areas`), опытные срезы (`EXPERIENCE_SHARDS`), лимит `DEFAULT_LIMIT`.
- Периодичность: ежедневный запуск (дельта активных вакансий), цель >500k строк.
- Ограничения hh.ru: лимит на выдачу, антибот, необходимость пауз и ротации User-Agent/proxy.
- Куда идёт дальше: поля из CSV мапятся на признаки из `docs/02_feature_dictionary_hh.md` (формат работы, грейд, роли, стек, бенефиты, англ/образование) и используются в пайплайне + продуктовых персонах.

In [ ]:

from pathlib import Path
import sys
import importlib
import pandas as pd

CWD = Path.cwd()
CANDIDATES = [CWD] + list(CWD.parents)
PROJECT_ROOT = next((p for p in CANDIDATES if (p / "src" / "skillra_pda").exists()), CWD)
SRC_DIR = PROJECT_ROOT / "src"

for path in {PROJECT_ROOT, SRC_DIR}:
    if str(path) not in sys.path:
        sys.path.insert(0, str(path))

from src.skillra_pda import config, io, cleaning, features, eda, viz, personas
from importlib import reload
for module in (cleaning, features, eda):
    reload(module)

config.ensure_directories()

if not config.CLEAN_DATA_FILE.exists() or not config.FEATURE_DATA_FILE.exists():
    import scripts.run_pipeline as run_pipeline
    run_pipeline.main()

raw_path = config.RAW_DATA_FILE
clean_path = config.CLEAN_DATA_FILE
feat_path = config.FEATURE_DATA_FILE

df_raw = io.load_raw(raw_path)
df_clean = pd.read_parquet(clean_path)
df_features = pd.read_parquet(feat_path)

df_features.head(2)


### Эволюция датасета
Прослеживаем размеры и ключевые доли на каждом этапе пайплайна: сырой CSV → очищенный parquet → фичи → агрегированная витрина `market_view`.

In [ ]:
from src.skillra_pda import market

market_view_path = config.PROCESSED_DATA_DIR / "market_view.parquet"
df_market_view = (
    pd.read_parquet(market_view_path)
    if market_view_path.exists()
    else market.build_market_view(df_features.copy())
)

def summarize_stage(name: str, df, weight_col: str | None = None):
    rows, cols = df.shape
    salary_cols = [c for c in ["salary_mid_rub_capped", "salary_mid", "salary_from", "salary_to"] if c in df]
    salary_share = df[salary_cols].notna().any(axis=1).mean() if salary_cols else None
    non_rub = ((df.get("currency", "RUB") != "RUB").mean()) if "currency" in df else None
    remote_share = None
    unknown_work = None
    if "work_mode" in df:
        work_mode_series = df["work_mode"].fillna("unknown").str.lower()
        remote_share = work_mode_series.isin(["remote", "hybrid"]).mean()
        unknown_work = work_mode_series.eq("unknown").mean()
    elif "is_remote" in df:
        remote_share = df["is_remote"].fillna(False).mean()
    elif "remote_share" in df:
        weights = df[weight_col] if weight_col and weight_col in df else None
        if weights is not None and weights.sum() > 0:
            remote_share = float((df["remote_share"] * weights).sum() / weights.sum())
        else:
            remote_share = df["remote_share"].mean()
    return {
        "stage": name,
        "rows": rows,
        "cols": cols,
        "salary_specified_share": salary_share,
        "non_rub_share": non_rub,
        "remote_or_hybrid_share": remote_share,
        "unknown_work_mode_share": unknown_work,
    }

evolution_rows = [
    summarize_stage("raw", df_raw),
    summarize_stage("clean", df_clean),
    summarize_stage("features", df_features),
    summarize_stage("market_view", df_market_view, weight_col="vacancy_count"),
]
evolution_df = pd.DataFrame(evolution_rows)
evolution_df

In [ ]:
from IPython.display import Markdown, display

evo = evolution_df.set_index("stage")
raw_stage = evo.loc["raw"]
clean_stage = evo.loc["clean"]
feat_stage = evo.loc["features"]
market_stage = evo.loc["market_view"]

def fmt_int(x: int) -> str:
    return f"{int(x):,}".replace(",", " ")

summary_md = Markdown(
    f"**Ключевые выводы по эволюции:** "
    f"n {fmt_int(raw_stage['rows'])} → {fmt_int(feat_stage['rows'])}, "
    f"колонок {fmt_int(raw_stage['cols'])} → {fmt_int(feat_stage['cols'])}; "
    f"доля строк с указанной зарплатной вилкой после clean — {clean_stage['salary_specified_share']:.1%}, "
    f"non-RUB в market_view — {market_stage['non_rub_share']:.1%}; "
    f"remote+hybrid на этапе features — {feat_stage['remote_or_hybrid_share']:.1%}, "
    f"work_mode=unknown — {feat_stage['unknown_work_mode_share']:.1%}."
)
display(summary_md)


### 0.1a Здоровье данных: NaN и маркеры unknown
Проверяем, где остаются пропуски и текстовые маркеры неопределённости на этапах raw → clean → features.

In [ ]:
health_raw = cleaning.summarize_data_health(df_raw, prefix="raw:")
health_clean = cleaning.summarize_data_health(df_clean, prefix="clean:")
health_features = cleaning.summarize_data_health(df_features, prefix="feat:")

def top_health(df: pd.DataFrame, n: int = 12):
    return df.sort_values(by=["share_nan", "share_unknown_marker"], ascending=False).head(n)

health_raw_top = top_health(health_raw)
health_clean_top = top_health(health_clean)
health_features_top = top_health(health_features)

health_raw_top, health_clean_top, health_features_top

In [ ]:
default_obj_clean = pd.Series(pd.NA, index=df_clean.index, dtype=object)
default_obj_feat = pd.Series(pd.NA, index=df_features.index, dtype=object)
default_bool_clean = pd.Series(pd.NA, index=df_clean.index, dtype="boolean")

work_mode_unknown_share = (
    df_features.get("work_mode", default_obj_feat)
    .fillna("unknown")
    .astype(str)
    .str.lower()
    .eq("unknown")
    .mean()
)
english_series = df_clean.get("lang_english_level", default_obj_clean)
english_unknown_share = (
    english_series
    .fillna("unknown")
    .astype(str)
    .str.lower()
    .eq("unknown")
    .mean()
)
english_required_series = df_clean.get("lang_english_required", default_bool_clean)
english_required_nan = english_required_series.isna().mean()
education_series = df_clean.get("education_level", default_obj_clean)
education_unknown_share = (
    education_series
    .fillna("unknown")
    .astype(str)
    .str.lower()
    .eq("unknown")
    .mean()
)
benefits_missing_share = df_clean.filter(like="benefit_").isna().mean().mean() if df_clean.filter(like="benefit_").shape[1] else 0
soft_missing_share = df_clean.filter(like="soft_").isna().mean().mean() if df_clean.filter(like="soft_").shape[1] else 0
health_key_figures = {
    "work_mode_unknown_share": work_mode_unknown_share,
    "english_unknown_share": english_unknown_share,
    "english_required_nan": english_required_nan,
    "education_unknown_share": education_unknown_share,
    "benefits_missing_share": benefits_missing_share,
    "soft_missing_share": soft_missing_share,
}
health_key_figures

In [ ]:
from IPython.display import Markdown, display

n_features = len(df_features)
grade_counts = df_features["grade"].fillna("unknown").astype(str).str.lower().value_counts()
work_mode_counts = df_features.get("work_mode", pd.Series(dtype=str)).fillna("unknown").astype(str).str.lower().value_counts()
english_unknown = (
    df_features.get("lang_english_level", pd.Series(dtype=str))
    .fillna("unknown")
    .astype(str)
    .str.lower()
    .eq("unknown")
    .mean()
)
salary_missing = df_features[["salary_from", "salary_to"]].isna().all(axis=1).mean()

coverage_md = Markdown(
    f"**Coverage ключевых полей:** выборка {n_features:,}. "
    f"unknown grade — {grade_counts.get('unknown', 0) / max(n_features, 1):.1%}, "
    f"unknown work_mode — {work_mode_counts.get('unknown', 0) / max(n_features, 1):.1%}, "
    f"unknown уровень английского — {english_unknown:.1%}, "
    f"пропусков зарплатной вилки — {salary_missing:.1%}. "
    f"Высокая доля неизвестных грейдов и форматов работы может смещать распределения."
)
display(coverage_md)


**Что увидели по качеству:**
- В raw крупные пропуски в адресных/образовательных полях (`metro_primary` 98%, `employer_accredited_it` 93%) и в верхней границе зарплаты (`salary_to` 47%).
- После cleaning текстовые маркеры неопределённости приводим к `NaN`, заполняем категориальные поля "unknown"; доли NaN у булевых признаков <1%.
- В фичах главная зона неопределённости — `work_mode`: 34.1% вакансий остаются без явного формата.
- Требования к английскому: 3.5% записей с unknown/NaN по уровню, флаг обязательности без пропусков; образование не указано в 100% вакансий, поэтому выводы про образование считаем непредставительными.
- Бенефиты/soft-skills остаются разреженными, но после нормализации нет NaN — считаем только явные единицы и не включаем их в ключевые доли.

### 0.2 Пример сырых данных и базовые статы
Показываем head, `info()` и первичный профиль пропусков/типов, чтобы понимать качество источника до очистки.

In [ ]:

import io

raw_head = df_raw.head(5)
buffer = io.StringIO()
df_raw.info(buf=buffer)
raw_info = buffer.getvalue()

raw_overview = {
    "raw_path": str(raw_path),
    "raw_shape": df_raw.shape,
    "columns_sample": df_raw.columns.tolist()[:10],
}
raw_profile = cleaning.basic_profile(df_raw)
raw_missing_top = eda.missing_share(df_raw, top_n=10)
raw_numeric_stats = df_raw.select_dtypes("number").describe().T

raw_head, raw_info, raw_overview, raw_profile, raw_missing_top, raw_numeric_stats


### 0.3 Выводы по этапу 0
- **Что сделали:** описали ежедневный парсер hh.ru, его фильтры/ограничения, показали сырой head и профиль данных.
- **Почему так:** ежедневная дельта и лимиты hh.ru требуют шардирования опыта и аккуратных пауз, чтобы собирать стабильный поток вакансий.
- **Ключевые цифры:** 7 026 строк и 151 колонка в raw, 100% строк содержат хотя бы одну границу зарплаты, 7.8% вакансий в USD/EUR.
- **Мини-чек-лист:** после парсинга имеем сырую выгрузку 7 026×151 без дублей; зафиксировали поля зарплаты/валюты как основу для очистки и конвертации.
- **Что это даёт Skillra:** стабильная точка входа данных для аналитики и рекомендации карьерных путей, масштабируемая до 500k+ вакансий.

## Этап 1. Предобработка (cleaning.py)
Повторяем шаги cleaning: парсинг дат, дедупликация, обработка пропусков, нормализация булевых колонок (`salary_gross` и все флаги is_/has_/benefit_/soft_/domain_/role_). Готовые результаты берём из `data/processed/hh_clean.parquet`. Ниже — снимки пропусков и типов.

In [ ]:

dup_raw = df_raw.duplicated().sum()
dup_clean = df_clean.duplicated().sum()
missing_salary = df_raw[[c for c in df_raw.columns if "salary" in c]].isna().mean().to_dict()

clean_missing_top = eda.missing_share(df_clean, top_n=10)
salary_gross_dtype = str(df_clean.get("salary_gross", pd.Series(dtype="boolean")).dtype)
salary_gross_stats = df_clean.get("salary_gross", pd.Series(dtype="boolean")).value_counts(dropna=False)

clean_snapshot = {
    "clean_shape": df_clean.shape,
    "salary_gross_dtype": salary_gross_dtype,
    "salary_gross_counts": salary_gross_stats.to_dict(),
    "duplicates_raw": int(dup_raw),
    "duplicates_clean": int(dup_clean),
    "missing_salary_share_raw": missing_salary,
}
clean_snapshot, clean_missing_top


### 1.1 Выводы по этапу 1
- **Что сделали:** привели типы, очистили пропуски, дедуплицировали, нормализовали зарплаты и каппинг.
- **Почему так:** без единых типов и контроля `salary_gross`/валюты нельзя честно сравнивать роли и города.
- **Ключевые цифры:** размер датасета сохранился 7 026×151, дублей нет (0→0); `salary_from` заполнен в 86% строк, `salary_to` — в 53%.
- **Мини-чек-лист:** до этапа 7 026×151 raw → после 7 026×151 clean; конвертировали валюту, привели булевые и дату публикации, подготовили `salary_mid_rub_capped`.
- **Что это даёт Skillra:** очищенные данные → стабильные витрины и корректные графики/персоны без шумовых выбросов.

## Этап 2. Признаки (features.py)
Используем `features.engineer_all_features`: свежесть вакансии (`vacancy_age_days`), география (`city_tier`), формат работы (`work_mode`), зарплатные бакеты (`salary_bucket`), стековые агрегаты (`core_data_skills_count`, `ml_stack_count`, `tech_stack_size`), текстовые метрики (`description_len_chars/words`), продуктовые флаги (`is_junior_friendly`, `battle_experience`).

In [ ]:

key_features = [
    "vacancy_age_days",
    "city_tier",
    "work_mode",
    "salary_bucket",
    "core_data_skills_count",
    "ml_stack_count",
    "tech_stack_size",
    "description_len_chars",
    "description_len_words",
    "is_junior_friendly",
    "battle_experience",
]
available = [c for c in key_features if c in df_features.columns]

feature_snapshot = df_features[available].head(5)
feature_stats = df_features[[c for c in available if pd.api.types.is_numeric_dtype(df_features[c])]].describe().T
feature_snapshot, feature_stats


### 2.1 Выводы по этапу 2
- **Что сделали:** добавили признаки work_mode, city_tier, salary_bucket, домены, роли, размер стека, junior-friendly, англ/образование.
- **Почему так:** эти фичи нужны для продукта (подбор ролей/треков) и для честного EDA по сегментам.
- **Ключевые цифры:** после feature engineering 7 026×164 (+13 фичей); work_mode даёт 57% remote/hybrid и 5% офис/field, но 34% строк в категории `unknown` — доли форматов могут быть смещены.
- **Мини-чек-лист:** до этапа 7 026×151 clean → после 7 026×164 features; добавили work_mode, city_tier, salary_bucket, стековые счётчики, primary_role, флаги junior-friendly/remote.
- **Что это даёт Skillra:** готовая витрина для подсказок пользователю и расчёта skill-gap по выбранному сегменту.

## Этап 3. Разведочный анализ (EDA)
Смотрим рынок по зарплатам, форматам работы, ролям, навыкам, доменам, английскому, образованию и работодателям. Используем агрегаты из `eda.py` и визуализации из `viz.py`; добавляем текстовые выводы для каждой темы.

In [ ]:
import importlib

import src.skillra_pda.eda as eda_module
eda = importlib.reload(eda_module)
if not hasattr(eda, "salary_by_city_tier"):
    raise AttributeError("expected salary_by_city_tier in eda module; ensure local code is on sys.path")

salary_city = eda.salary_by_city_tier(df_features)
salary_grade = eda.salary_by_grade(df_features)
salary_role = eda.salary_by_primary_role(df_features)
salary_stack = eda.salary_by_stack_size(df_features)

skill_cols = [c for c in df_features.columns if c.startswith("skill_") or c.startswith("has_")]
top_skills = eda.skill_frequency(df_features, skill_cols, top_n=15).index.tolist()
skill_share = eda.skill_share_by_grade(df_features, top_skills)

benefits_grade = eda.benefits_summary_by_grade(df_features)
soft_corr = eda.soft_skills_correlation(df_features)
junior_roles = eda.junior_friendly_share(df_features, group_col="primary_role")

salary_city.head(), salary_grade.head(), salary_role.head(), salary_stack.head(), skill_share.head(), benefits_grade.head(), soft_corr.head(), junior_roles.head()


### 3.1 Рынок по доменам (отраслям)
Используем доменные флаги (`domain_*`), чтобы увидеть распределение вакансий и зарплат по ключевым отраслям. Приоритеты строим по частоте, чтобы корректно выбрать главный домен при нескольких флагах.

In [ ]:
domain_cols = [c for c in df_features.columns if c.startswith("domain_")]
domain_priorities = (
    df_features[domain_cols]
    .sum()
    .sort_values(ascending=False)
    .index.str.removeprefix("domain_")
    .tolist()
)


In [ ]:
from src.skillra_pda import viz
viz = importlib.reload(viz)

fig_salary_domain, salary_domain_path = viz.salary_by_domain_plot(
    df_features, top_n=10, priorities=domain_priorities, return_fig=True
)

In [ ]:
domain_salary = eda.describe_salary_by_domain(df_features, priorities=domain_priorities)
domain_salary.head(10)

In [ ]:
from IPython.display import Markdown, display

main_domains = domain_salary.head(5)
unknown_share_series = domain_salary.loc[domain_salary['domain'] == 'unknown', 'share']
unknown_share = float(unknown_share_series.iloc[0]) if not unknown_share_series.empty else 0
market_n = int(domain_salary['vacancy_count'].sum())
domain_lines = [
    f"Финтех — {main_domains.iloc[1]['share']:.1%} выборки с медианой {main_domains.iloc[1]['salary_median']:.0f} ₽",
    f"Продуктовые IT — {main_domains.iloc[2]['share']:.1%} с медианой {main_domains.iloc[2]['salary_median']:.0f} ₽",
]

domain_md = Markdown(f"""
#### Выводы по доменам
n={market_n:,} вакансий; топ-2 домена: {domain_lines[0]}, {domain_lines[1]}. Unknown домен — {unknown_share:.1%}, поэтому зарплаты по отраслям трактуем осторожно.
""")
display(domain_md)


### 3.2 Зарплаты и формат работы
Комбинируем грейд×город и роль×формат, чтобы увидеть премии за локацию и удалёнку.

#### 3.2.1 Зарплаты по грейду и типу города
Медианы по грейду/локации: senior — 275k в Москве против 178k в регионах, middle — 230k против 185k; стажёры держатся на ~67k в Москве и 60k в регионах. География объёма: Москва/СПб дают 56% выборки, миллионники — ещё 10%.

In [ ]:
grade_order = ["intern", "junior", "middle", "senior", "lead", "architect", "unknown"]
grade_city_summary = eda.salary_summary_by_grade_and_city(df_features)
grade_city_summary["grade"] = pd.Categorical(grade_city_summary["grade"], categories=grade_order, ordered=True)
grade_city_summary = grade_city_summary.sort_values(["grade", "city_tier"])
fig_salary_grade_city, grade_city_path = viz.salary_by_grade_city_heatmap(
    grade_city_summary, return_fig=True
)


In [ ]:
grade_city_pivot = grade_city_summary.pivot(index="grade", columns="city_tier", values="median")
city_tier_share = df_features["city_tier"].value_counts(normalize=True)
salary_spread = {
    "senior_moscow": grade_city_pivot.loc["senior", "Moscow"],
    "senior_regions": grade_city_pivot.loc["senior", "Other RU"],
    "middle_moscow": grade_city_pivot.loc["middle", "Moscow"],
    "middle_regions": grade_city_pivot.loc["middle", "Other RU"],
    "intern_moscow": grade_city_pivot.loc["intern", "Moscow"],
}
city_tier_share, pd.DataFrame([salary_spread])

*График: тепловая карта зарплат по грейду и типу города.*
- **Выводы про рынок:** senior остаются с премией даже в регионах, Москва/СПб дают максимальные медианы для всех грейдов.
- **Что это даёт Skillra:** можем объяснять кандидату, как город влияет на ожидания по зарплате для его грейда и стоит ли рассматривать релокацию.

#### 3.2.1a Дополнительные salary-срезы (объём + медиана)
На тепловых картах видно, что основные зарплатные объёмы сосредоточены в Москве и крупных городах (≈66% вакансий), премия по грейду сохраняется даже при узких выборках.

In [ ]:
fig_salary_city_counts, salary_city_path = viz.salary_by_city_mean_count_plot(
    df_features, top_n=8, return_fig=True
)

График: медианные зарплаты и объём вакансий по типу города.
- **Рынок:** Москва/1-й эшелон дают львиную долю спроса и более высокие медианы.
- **Skillra:** можно приоритизировать рекомендации по топ-городам и показывать разницу удалёнки vs офис.

In [ ]:
fig_salary_grade_counts, salary_grade_counts_path = viz.salary_by_grade_mean_count_plot(
    df_features, return_fig=True
)

График: медианные зарплаты и объём вакансий по грейдам.
- **Рынок:** наибольший спрос и устойчивые зарплаты на middle, но заметна премия senior.
- **Skillra:** продукт может объяснять разницу ожиданий по грейдам и планировать апскилл для перехода middle→senior.

In [ ]:
fig_salary_role_counts, salary_role_counts_path = viz.salary_by_primary_role_mean_count_plot(
    df_features, top_n=10, return_fig=True
)

График: зарплаты и объём по топ-ролям.
- **Рынок:** выше всего медианы у ML/DS, но объём максимален у product/data-ролей.
- **Skillra:** приоритетные треки — data/product, но с пояснением про конкуренцию и требования по стеку.

In [ ]:
fig_salary_stack_bucket, salary_stack_bucket_path = viz.salary_by_skills_bucket_plot(
    df_features, return_fig=True
)

График: зарплаты в зависимости от размера стека.
- **Рынок:** рост зарплаты коррелирует с шириной стека до 6–10 технологий, далее эффект плато.
- **Skillra:** в рекомендациях Skillra можно подсветить «оптимальный» стек для роста без избыточного перегруза.

#### 3.2.2 Зарплаты по ролям и формату работы
Удалёнка/гибрид в сумме дают 57% вакансий, офис/field — около 9%; у ML/DevOps/Backend доля remote 71–81%, что отражается на премиях в тепловой карте. 34% work_mode = `unknown`, поэтому итоговые доли форматов стоит трактовать осторожно.

In [ ]:
role_work_summary = eda.salary_summary_by_role_and_work_mode(df_features)
role_work_pivot = role_work_summary.pivot(index="primary_role", columns="work_mode", values="salary_median")
fig_salary_role_workmode, role_workmode_path = viz.salary_by_role_work_mode_heatmap(
    role_work_summary, return_fig=True
)
role_work_summary.head(), role_work_pivot

*График: зарплаты по ролям и формату работы.*
- **Выводы про рынок:** гибрид у продуктовых и аналитических ролей сопоставим с офисом; backend/ML на полном remote часто имеют дисконт.
- **Что это даёт Skillra:** при подборе ролей подсвечиваем, сколько можно ожидать на remote/office и стоит ли искать гибрид ради зарплатной премии.

#### 3.2.3 Доля удалёнки и junior-friendly по ролям
Удалёнка лидирует в ML (81%), backend (71%) и frontend (75%); junior-friendly выше у mobile/other (32–44%) и data-ролей (~30%). Из-за 34% строк с `unknown` work_mode оценки долей удалёнки могут быть смещены.

In [ ]:
remote_share = eda.remote_share_by_role(df_features)
junior_roles = eda.junior_friendly_share(df_features, group_col="primary_role")
fig_remote_share, remote_share_path = viz.remote_share_by_role_bar(remote_share, return_fig=True)
remote_share.head(), junior_roles.head()

In [ ]:
work_mode_share = df_features["work_mode"].value_counts(normalize=True, dropna=False)
remote_hybrid_share = work_mode_share.get("remote", 0) + work_mode_share.get("hybrid", 0)
unknown_work_mode_share = work_mode_share.get("unknown", 0)
work_mode_share, remote_hybrid_share, unknown_work_mode_share

In [ ]:
from IPython.display import Markdown, display

remote_hybrid_share = work_mode_share.get("remote", 0) + work_mode_share.get("hybrid", 0)
unknown_work_mode_share = work_mode_share.get("unknown", 0)
work_mode_md = Markdown(
    f"**Форматы работы:** remote+hybrid — {remote_hybrid_share:.1%}, офис — {work_mode_share.get('office', 0):.1%}, field — {work_mode_share.get('field', 0):.1%}, unknown — {unknown_work_mode_share:.1%}. "
    "Высокая доля unknown (треть выборки) смещает выводы по форматам."
)
display(work_mode_md)


### 3.3 Топ работодатели, бенефиты и soft-skills
Определяем лидеров по количеству вакансий и проверяем, какие бенефиты и мягкие навыки они чаще всего упоминают.

In [ ]:
top_employers_df = eda.top_employers(df_features)

top_employers_df.head(10)

In [ ]:
fig_benefits_employer, benefits_path = viz.benefits_employer_heatmap(
    df_features, top_n_employers=12, top_n_benefits=12, return_fig=True
)

График: бенефиты у топ-работодателей.
- **Рынок:** ДМС, релокация и обучение чаще у крупных работодателей.
- **Skillra:** можно рекомендовать компании с подходящими пакетами и объяснять, где искать ДМС/релокацию.

In [ ]:
fig_soft_employer, soft_path = viz.soft_skills_employer_heatmap(
    df_features, top_n_employers=12, top_n_skills=12, return_fig=True
)

График: soft skills у топ-работодателей.
- **Рынок:** коммуникация, аналитическое мышление и командная работа встречаются чаще всего.
- **Skillra:** объясняем пользователям, какие soft skills нужны для входа в топ-компании.

#### 3.3.1 Топ навыков для data-ролей

In [ ]:
skill_cols = [c for c in df_features.columns if c.startswith("skill_")]
fig_top_skills, top_skills_path = viz.top_skills_bar(
    df_features, skill_cols=skill_cols, role_filter="data", top_n=12, return_fig=True
)

In [ ]:
data_roles_mask = df_features["primary_role"].str.contains("data", case=False, na=False)
data_skill_freq = df_features.loc[data_roles_mask, skill_cols].sum().sort_values(ascending=False).head(10)
data_role_count = int(data_roles_mask.sum())
data_role_count, data_skill_freq

In [ ]:
from IPython.display import Markdown, display

data_top = data_skill_freq.head(5)
skill_lines = [f"{skill.replace('skill_', '').replace('_', ' ')} — {count / data_role_count:.1%} (n={int(count)})" for skill, count in data_top.items()]
skill_md = Markdown(f"""
Data-вакансий: {data_role_count}.
Топ-5 хард-скиллов (доля строк):
{'<br>'.join(skill_lines)}
Высокая доля unknown work_mode={work_mode_share.get('unknown', 0):.1%} может смещать оценки сегмента.
""")
display(skill_md)


*Data-рынок ожидаемо требует SQL, Python, Airflow и BI-инструменты: они формируют базовый корсет для DA/DE. 
Эти навыки должны быть в приоритете рекомендаций Skillra (курсы, воркшопы, вакансии).*

#### Выводы по работодателям
- Топ-10 компаний покрывают 12.7% выборки: от поддержки (Яндекс Крауд, 2.1% вакансий, медиана 33k) до интеграторов с медианой 225k — важно смотреть на стек и роль.
- Часто встречающиеся бенефиты: ДМС, гибкий график, возможность удалёнки/релокации.
- В soft-skills топе — коммуникация и самоорганизация; на них стоит делать акцент при подготовке кандидатов.

### 3.4 Английский и образование
Смотрим требования к языку и образованию: распределения долей, премии по зарплатам и их интерпретация.

In [ ]:
req_df = df_features.copy()
for col in ["lang_english_required", "edu_required", "edu_technical"]:
    if col in req_df:
        req_df[col] = req_df[col].fillna(False)

english_stats = eda.english_requirement_stats(req_df)
education_stats = eda.education_requirement_stats(req_df)
english_unknown_share = 1 - english_stats["share"].sum()
education_unknown_share = 1 - education_stats["share"].sum()
english_stats, education_stats, english_unknown_share, education_unknown_share

In [ ]:
fig_salary_english, salary_english_path = viz.salary_by_english_level_plot(req_df, return_fig=True)

График: зарплаты по требуемому уровню английского.
- **Рынок:** явное требование английского встречается редко (B2+ всего 2.3%), медианы в этих сегментах 150–185k, но из-за малого объёма и 7.8% строк без уровня выводы требуют осторожности.
- **Skillra:** мотивируем улучшать английский для доступа к верхним вилкам и международным командам, но отмечаем низкую полноту метки.

In [ ]:
fig_salary_education, salary_edu_path = viz.salary_by_education_level_plot(req_df, return_fig=True)

График: зарплаты по требованиям к образованию.
- **Рынок:** сильной премии за формальное образование нет, важнее навыки и релевантный опыт.
- **Skillra:** упор на доказуемые навыки и проекты, а не на диплом; это снижается барьер входа для свитчеров.

In [ ]:
from IPython.display import Markdown, display

english_unknown_share = 1 - english_stats["share"].sum()
education_unknown_share = 1 - education_stats["share"].sum()
no_english_share = english_stats.loc[english_stats['english_level'] == 'no_english', 'share'].iat[0]
b2plus_share = english_stats.loc[english_stats['english_level'].isin(['B2', 'C1_plus'])]['share'].sum()
no_degree_share = education_stats.loc[education_stats['education_level'] == 'no_degree_required', 'share'].iat[0]
tech_degree_share = education_stats.loc[education_stats['education_level'] == 'technical_only', 'share'].iat[0]
any_degree_share = education_stats.loc[education_stats['education_level'] == 'any_degree', 'share'].iat[0]

req_md = Markdown(f"""
#### Выводы по требованиям
- Английский не требуется в {no_english_share:.1%} вакансий; уровни B2+ встречаются в {b2plus_share:.1%}. Unknown уровень — {english_unknown_share:.1%}, поэтому премии нужно трактовать аккуратно.
- Образование: без диплома {no_degree_share:.1%}, техническое высшее {tech_degree_share:.1%}, любое высшее {any_degree_share:.1%}; неизвестно — {education_unknown_share:.1%}.
- Эти требования сегментируют рынок, их стоит учитывать в продуктовых фильтрах.
""")
display(req_md)


### 3.5 Корреляционный анализ числовых признаков
Используем готовые функции `eda.correlation_matrix` и `viz.corr_heatmap`, чтобы понять связи между зарплатой и числовыми фичами.

In [ ]:
corr_cols = [
    col
    for col in [
        "salary_from",
        "salary_to",
        "vacancy_age_days",
        "tech_stack_size",
        "core_data_skills_count",
        "ml_stack_count",
        "description_len_words",
    ]
    if col in df_features.columns
]
correlations = eda.correlation_matrix(df_features, cols=corr_cols)
corr_fig, corr_fig_path = viz.corr_heatmap(correlations, return_fig=True)

*Связь с зарплатой:* `salary_from` и `salary_to` ожидаемо коррелируют между собой, негативная связь с `vacancy_age_days` подчёркивает, что свежие вакансии быстрее набирают отклики.

*Выводы для рынка:* размер стека и длина описания умеренно положительно связаны с зарплатами.

*Skillra:* можно использовать эти связи в подсказках по улучшению резюме и в приоритизации вакансий для пользователей.

### 3.6 Инсайты EDA
- **Что мы сделали:** построили тепловые карты по зарплатам (грейд×город, роль×формат), разложили долю удалёнки, выгрузили топ работодателей/бенефитов/soft-skills, посмотрели корреляции.
- **Почему именно так:** инвесторам и пользователям важны премии по локации/формату и качество работодателей; корреляции показывают, какие числовые признаки двигают зарплату.
- **Основные выводы про рынок:** Москва/СПб дают ~56% вакансий и медиану до 275k для senior; удалёнка/гибрид присутствует в 57% вакансий (ML ~81% remote) при 34% `unknown` work_mode; финтех 17% и IT-продукты 12% держат премию, но 54% строк без домена; английский явно требуют лишь 15% вакансий, аналогично образование.
- **Что это даёт продукту Skillra:** формируем подсказки по целевым кластерам (например, data-ролям в крупных городах или remote fintech) и готовим аргументы для карьерного трекинга.
- **Мини-чек-лист:** EDA проходит на датасете 7 026×164 без изменения формы; собрали числовые сводки по зарплатам/форматам/ролям, долям remote и junior-friendly, топам работодателей и навыков.

## Этап 4. Визуализации и сводка
Используем готовые функции из `viz.py`, сохраняем графики в `reports/figures`. Графики покрывают зарплаты, форматы работы, навыковые тепловые карты, soft-skills и распределения.

In [ ]:
fig_salary_role, salary_role_path = viz.salary_by_role_box(df_features, return_fig=True)

Галерея: распределение зарплат по ролям.
- **Рынок:** разброс внутри ролей велик, но медианы держатся выше у ML/DS.
- **Skillra:** помогает пользователю понять вилки внутри выбранной роли и оценить ожидания.

In [ ]:
fig_workmode, workmode_path = viz.work_mode_share_by_city(df_features, return_fig=True)

Галерея: формат работы по типу города.
- **Рынок:** удалёнка концентрируется в столицах, в регионах преобладает офис/гибрид.
- **Skillra:** можно подсказывать форматы работы и релокацию/удалёнку в зависимости от города пользователя.

In [ ]:
fig_skill_heatmap, skill_heatmap_path = viz.heatmap_skills_by_grade(skill_share, return_fig=True)

Галерея: навыки по грейдам.
- **Рынок:** у middle/senior шире стек и выше доля продвинутых skills.
- **Skillra:** подсвечиваем пользователю, какие навыки добавлять, чтобы перейти в следующий грейд.

### 4.1 Выводы по графикам
- **Что мы сделали:** собрали набор готовых графиков (боксплоты, bar+count, тепловые карты) прямо из кода `viz.py` и встроили в ноутбук.
- **Почему именно так:** хотим, чтобы ноутбук был self-contained и понятен инвестору: каждый график сразу отображается и сохраняется в `reports/figures`.
- **Основные выводы про рынок:** медианы и объёмы видны на одном экране, тепловые карты подчёркивают премии по локации/формату.
- **Что это даёт продукту Skillra:** готовы reusable-компоненты для веб-интерфейса (дашборд рынка) и презентаций.
- **Мини-чек-лист:** исходный датасет 7 026×164 остаётся неизменным, добавлены готовые артефакты в `reports/figures` (зарплаты по ролям, формат работы, heatmap навыков).
- **К какому следующему шагу это подводит:** переходим к персонализации (персоны) и масштабированию данных.

## Продуктовая часть: персоны и skill-gap
Персоны из контекста Skillra: студент (Junior DA/DS), свитчер в BI/продукт, middle аналитик. Используем `personas.analyze_persona` как основной API: он возвращает market summary, skill-gap и рекомендованные навыки.

In [ ]:
personas_mod = importlib.reload(personas)
persona_objects = [
    personas_mod.DATA_STUDENT,
    personas_mod.SWITCHER_BI,
    personas_mod.MID_DATA_ANALYST,
]

persona_reports = {}
for p in persona_objects:
    analysis = personas_mod.analyze_persona(df_features, p, top_k=10)
    persona_reports[p.name] = analysis

    persona_profile = pd.DataFrame({
        "persona": [p.name],
        "description": [p.description],
        "goals": [", ".join(p.goals) if p.goals else "—"],
        "limitations": [", ".join(p.limitations) if p.limitations else "—"],
        "target_role": [p.target_role],
        "target_grade": [p.target_grade],
        "target_city_tier": [p.target_city_tier],
        "target_work_mode": [p.target_work_mode],
    })
    display(persona_profile)

    summary_df = pd.DataFrame([analysis["market_summary"]])
    display(summary_df)

    top_demand = analysis.get("top_skill_demand")
    if isinstance(top_demand, pd.DataFrame) and not top_demand.empty:
        display(top_demand.head(5))

    display(analysis["skill_gap"].head(10))
    top_reco = analysis["recommended_skills"][:5]
    print(f"Рекомендуем добрать: {', '.join(top_reco) if top_reco else 'нет явных гэпов'}")
    print('-' * 40)


In [ ]:
gap_student = persona_reports.get("data_student", {}).get("skill_gap")
fig_gap_student, path_gap_student = personas_mod.plot_persona_skill_gap(
    gap_student, personas_mod.DATA_STUDENT, return_fig=True
)

In [ ]:
from IPython.display import Markdown, display

student_report = persona_reports.get("data_student", {})
student_gap = student_report.get("skill_gap", pd.DataFrame()).head(3)
student_market = student_report.get("market_summary", {})
student_skills = [row.skill_name for row in student_gap.itertuples()] if not student_gap.empty else []
student_md = Markdown(
    f"Skill-gap для персоны *data_student* (n={student_market.get('vacancy_count', 0)}): "
    + (", ".join(student_skills) if student_skills else "нет выраженных гэпов")
    + ". Remote доля — "
    + (f"{student_market.get('remote_share', 0):.1%}" if 'remote_share' in student_market else "—")
    + ", junior-friendly — "
    + (f"{student_market.get('junior_friendly_share', 0):.1%}" if 'junior_friendly_share' in student_market else "—")
    + "."
)
display(student_md)


In [ ]:
gap_switcher = persona_reports.get("switcher_bi", {}).get("skill_gap")
fig_gap_switcher, path_gap_switcher = personas_mod.plot_persona_skill_gap(
    gap_switcher, personas_mod.SWITCHER_BI, return_fig=True
)

In [ ]:
from IPython.display import Markdown, display

bi_report = persona_reports.get("switcher_bi", {})
bi_gap = bi_report.get("skill_gap", pd.DataFrame()).head(3)
bi_market = bi_report.get("market_summary", {})
bi_skills = [row.skill_name for row in bi_gap.itertuples()] if not bi_gap.empty else []
bi_md = Markdown(
    f"Skill-gap для персоны *switcher_bi* (n={bi_market.get('vacancy_count', 0)}): "
    + (", ".join(bi_skills) if bi_skills else "нет выраженных гэпов")
    + f". Remote+hybrid — {bi_market.get('remote_share', 0):.1%}, junior-friendly — {bi_market.get('junior_friendly_share', 0):.1%}."
)
display(bi_md)


*Визуализация skill-gap даёт готовый виджет для личного кабинета Skillra: пользователь видит, какие навыки закрывают 
максимальную долю вакансий в целевом сегменте, и может кликнуть в рекомендации курсов/проектов.*

In [ ]:
from IPython.display import Markdown, display

student_n = persona_reports.get("data_student", {}).get("market_summary", {}).get("vacancy_count", 0)
switcher_n = persona_reports.get("switcher_bi", {}).get("market_summary", {}).get("vacancy_count", 0)
mid_n = persona_reports.get("mid_data_analyst", {}).get("market_summary", {}).get("vacancy_count", 0)
summary_md = Markdown(f"""
### Выводы по персонам
- Студент DA/DS: сегмент n={student_n}, ключевые гэпы — топ-3 навыка из графика; remote {persona_reports['data_student']['market_summary'].get('remote_share', 0):.1%}.
- Свитчер BI: n={switcher_n}, приоритеты из графика; remote {persona_reports['switcher_bi']['market_summary'].get('remote_share', 0):.1%}.
- Mid аналитик: n={mid_n}, опираемся на видимые гэпы; выводы делаем только по отображённым навыкам.
""")
display(summary_md)


In [ ]:
from IPython.display import Markdown, display

notes_md = Markdown("""
### Пояснения по персонам
- Все выводы основаны на графике skill-gap: обсуждаем только навыки, которые реально в топе гэпов.
- n сегментов явно указано; при уменьшении выборки ниже min_market_n анализ не проводим.
- Высокая доля unknown по грейдам/форматам учитывается при интерпретации.
""")
display(notes_md)


## Итоговые выводы и чек-лист
### Выводы про рынок
- Зарплаты растут по грейду и укрупнённости города; в Москве медиана senior ~275k против 178k в регионах, удалёнка концентрируется в ролях data/ML (до 81%).
- Доменные лидеры (финтех ~17%, продуктовые IT ~12%) и топ работодатели держат медианы выше среднего, но 54% вакансий без домена — учитываем смещение.
- Требования к английскому/образованию редко жёсткие (только ~15–18% выборки с явными требованиями), поэтому рынок открыт для широкого пула кандидатов.
- Навыковая карта подтверждает спрос на SQL/BI для junior и продуктовых аналитиков, ML/infra — для senior; это отражено в skill-gap персонам.
- Корреляции показывают влияние размера стека и насыщенности описания на уровень вилки, что можно использовать в скоринге вакансий.

### Выводы для Skillra
- Готовые витрины (`hh_clean`, `hh_features`, `market_view`) позволяют строить карьерные рекомендации без ручной очистки.
- API `analyze_persona` даёт готовый блок для пользовательских сценариев (оценка сегмента, gap, next best skill).
- Самые ценные сегменты для продукта — data/ML и аналитика с высоким спросом на удалёнку и junior-friendly роли.
- Карта бенефитов/soft-skills по работодателям помогает подбирать целевые компании под цели пользователя.

### Чек-лист выполнения ТЗ
| Этап | Что сделано | Секция ноутбука |
| --- | --- | --- |
| 0. Данные | Парсер HH, raw head/info, профиль пропусков | Этап 0 |
| 1. Предобработка | Пропуски/булевы/дубликаты, `missing_share` | Этап 1 |
| 2. Признаки | `city_tier`, `work_mode`, стек, тексты, junior-friendly | Этап 2 |
| 3. EDA | Зарплаты, форматы, роли, навыки, домены, язык, образование, работодатели, корреляции | Этап 3 |
| 4. Визуализация | Графики зарплат/навыков/soft-skills/распределений | Этап 4 |
| Продуктовый слой | `analyze_persona`, skill-gap, рекомендации | Персоны |